In [ ]:
import mlflow
import mlflow.azureml

import azureml.core

print('SDK version:', azureml.core.VERSION)
print('MLflow version:', mlflow.version.VERSION)

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
import git

# get root of git repo
prefix = git.Repo('.', search_parent_directories=True).working_tree_dir

# project settings
project_uri = prefix+'/code/mlprojects/pytorch-mnist'

# azure ml settings
experiment_name = 'pytorch-mnist-mlflow-project-example'
compute_target = 'gpu-cluster'

In [ ]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [ ]:
mlflow.set_experiment(experiment_name)

In [ ]:
backend_config = {'COMPUTE': compute_target, 'USE_CONDA': False}

In [ ]:
run = mlflow.projects.run(uri=project_uri, 
                          backend='azureml',
                          backend_config=backend_config)

In [ ]:
run 

## Deploy model

In [ ]:
from azureml.core.webservice import AciWebservice, AksWebservice, Webservice

model_path = 'model'

aks_config = AksWebservice.deploy_configuration(compute_target_name='aks-cpu-deploy',
                                                cpu_cores=2,
                                                memory_gb=5,
                                                tags={'data': 'MNIST', 'method': 'pytorch'}, 
                                                description='Predict using webservice')

webservice, azure_model = mlflow.azureml.deploy(model_uri='runs:/{}/{}'.format(run.run_id, model_path),
                                                      workspace=ws,
                                                      deployment_config=aks_config,
                                                      service_name='pytorch-mnist-example',
                                                      model_name='pytorch-mnist-example')

## Test webservice

In [ ]:
from torchvision import datasets, transforms
import random
import numpy as np

test_data = datasets.MNIST('../../data/tmp', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))]), download=True)


def get_random_image():
    image_idx = random.randint(0,len(test_data))
    image_as_tensor = test_data[image_idx][0]
    return {'data': elem for elem in image_as_tensor.numpy().reshape(1,1,-1).tolist()}

In [ ]:
%matplotlib inline

import json
import matplotlib.pyplot as plt

test_image = get_random_image()

response = webservice.run(json.dumps(test_image))

response = sorted(response[0].items(), key = lambda x: x[1], reverse = True)


print('Predicted label:', response[0][0])
plt.imshow(np.array(test_image['data']).reshape(28,28), cmap = 'gray')

## Clean up
You can delete the AKS deployment with a delete API call.

In [ ]:
webservice.delete()